In [1]:
import numpy as np
from datetime import datetime

# 수치미분 함수

def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = tmp_val - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad

# sigmoid 함수

def sigmoid(x):
    return 1 / (1+np.exp(-x))

In [2]:
# LogicGate Class

class LogicGate:
    
    # 생성자
    # xdata, tdata => numpy.array(...)
    def __init__(self, gate_name, xdata, tdata, input_nodes, hidden_1_nodes, output_nodes, learning_rate, iteration_count):
        
        self.name = gate_name
        
        # 입력 데이터, 정답 데이터 초기화
        self.xdata = xdata.reshape(4, 2)
        self.tdata = tdata.reshape(4, 1)
        
        # 2층 hidden layer unit : 8 개 가정
        # 가중치 W, 바이어스 b 초기화
        self.W2 = np.random.rand(input_nodes, hidden_1_nodes)  
        self.b2 = np.random.rand(hidden_1_nodes)
        
        # 3층 output layer unit : 1 개 
        self.W3 = np.random.rand(hidden_1_nodes, output_nodes)
        self.b3 = np.random.rand(output_nodes)
                        
        # 학습률 learning rate 초기화
        self.learning_rate = learning_rate
        
        # 반복횟수 초기화
        self.iteration_count = iteration_count
    
        print(self.name + " object is created")
        
    # 손실함수
    def feed_forward(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z2 = np.dot(self.xdata, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        y = a3 = sigmoid(z3)
    
        # cross-entropy 
        return  -np.sum( self.tdata*np.log(y + delta) + (1-self.tdata)*np.log((1 - y)+delta ) )
    
    # obtain W and b
    def get_W_b(self):
        
        return self.W2,  self.b2, self.W3, self.b3
    
    # 손실 값 계산
    def loss_val(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z2 = np.dot(self.xdata, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        y = a3 = sigmoid(z3)
    
        # cross-entropy 
        return  -np.sum( self.tdata*np.log(y + delta) + (1-self.tdata)*np.log((1 - y)+delta ) )
    
    # query, 즉 미래 값 예측 함수
    def predict(self, xdata):
        
        z2 = np.dot(xdata, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        y = a3 = sigmoid(z3)
    
        if y >= 0.5:
            result = 1  # True
        else:
            result = 0  # False
    
        return y, result

    # accuracy method 1
    def accuracy1(self, test_xdata, test_tdata):
        
        matched_list = []
        not_matched_list = []
        
        for index in range(len(test_xdata)):
            
            (real_val, logical_val) = self.predict(test_xdata[index])
            
            if logical_val == test_tdata[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                
        accuracy_result = len(matched_list) / len(test_xdata)
        
        print("Accuracy => ", accuracy_result)
        
        return matched_list, not_matched_list
    
    # accuracy method 2
    def accuracy2(self, test_data):
        
        matched_list = []
        not_matched_list = []
        
        input_data = test_data[ :, 0:-1]
        target_data = test_data[ :, -1]
        
        for index in range(len(input_data)):
            
            (real_val, logical_val) = self.predict(input_data[index])
            
            if logical_val == target_data[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                
        accuracy_result = len(matched_list) / len(input_data)
        
        print("Accuracy => ", accuracy_result)
        
        return matched_list, not_matched_list
        
    # 수치미분을 이용하여 손실함수가 최소가 될때 까지 학습하는 함수
    def train(self):
        
        f = lambda x : self.feed_forward()
        
        print("Initial loss value = ", self.loss_val())
        
        start_time = datetime.now()
        
        for step in  range(self.iteration_count):
            
            self.W2 -= self.learning_rate * numerical_derivative(f, self.W2)
    
            self.b2 -= self.learning_rate * numerical_derivative(f, self.b2)
        
            self.W3 -= self.learning_rate * numerical_derivative(f, self.W3)
    
            self.b3 -= self.learning_rate * numerical_derivative(f, self.b3)
    
            if (step % 400 == 0):
                print("step = ", step, "loss value = ", self.loss_val())
                
        end_time = datetime.now()
        
        print("")
        print("Elapsed Time => ", end_time - start_time)

In [3]:
and_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
and_tdata = np.array([0, 0, 0, 1])

input_nodes = 2
hidden_1_nodes = 8
output_nodes = 1

lr = 1e-2  # learning rate
count = 10001  # iteration count

# AND Gate 객체 생성 및 training
and_obj = LogicGate("AND", and_xdata, and_tdata, input_nodes, hidden_1_nodes, output_nodes, lr, count)

and_obj.train()

AND object is created
Initial loss value =  11.073770813049723
step =  0 loss value =  10.649554791527581
step =  400 loss value =  2.2173475036588783
step =  800 loss value =  2.104900931285025
step =  1200 loss value =  1.9542133173581324
step =  1600 loss value =  1.7514049082883965
step =  2000 loss value =  1.5007962627134424
step =  2400 loss value =  1.2263773691632471
step =  2800 loss value =  0.9539520652064089
step =  3200 loss value =  0.7010585977654185
step =  3600 loss value =  0.5033410749286051
step =  4000 loss value =  0.3763729362050432
step =  4400 loss value =  0.2939246765399278
step =  4800 loss value =  0.2367845560295218
step =  5200 loss value =  0.19538127352264292
step =  5600 loss value =  0.16443055536293005
step =  6000 loss value =  0.14070728793016402
step =  6400 loss value =  0.12212927957780165
step =  6800 loss value =  0.10730410887590794
step =  7200 loss value =  0.0952753057767246
step =  7600 loss value =  0.08537054871083155
step =  8000 loss

In [4]:
# AND Gate prediction
print(and_obj.name, "\n")

test_data = np.array([ [0, 0, 0], [0, 1, 0], [1, 0, 0], [1, 1, 1] ])

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 0, 0, 1])

(true_list, false_list) = and_obj.accuracy1(test_xdata, test_tdata)
(true_list, false_list) = and_obj.accuracy2(test_data)

AND 

Accuracy =>  1.0
Accuracy =>  1.0


In [5]:
or_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
or_tdata = np.array([0, 1, 1, 1])

input_nodes = 2
hidden_1_nodes = 8
output_nodes = 1

lr = 1e-2  # learning rate
count = 10001  # iteration count

# OR Gate 객체 생성 및 training
or_obj = LogicGate("OR", or_xdata, or_tdata, input_nodes, hidden_1_nodes, output_nodes, lr, count)

or_obj.train()

OR object is created
Initial loss value =  2.6750072082441365
step =  0 loss value =  2.652082157263534
step =  400 loss value =  1.9548971614837265
step =  800 loss value =  1.7684575848007649
step =  1200 loss value =  1.506347530950729
step =  1600 loss value =  1.1838079674818154
step =  2000 loss value =  0.8689287327974127
step =  2400 loss value =  0.6208593742242137
step =  2800 loss value =  0.4485218718380379
step =  3200 loss value =  0.3338265139447598
step =  3600 loss value =  0.2570583390310509
step =  4000 loss value =  0.2042953509851337
step =  4400 loss value =  0.16683951721685444
step =  4800 loss value =  0.13939446091816268
step =  5200 loss value =  0.11869883606006473
step =  5600 loss value =  0.10269383218012335
step =  6000 loss value =  0.09004171274840156
step =  6400 loss value =  0.07984788596517914
step =  6800 loss value =  0.07149768978349061
step =  7200 loss value =  0.06455818941040808
step =  7600 loss value =  0.05871751917628494
step =  8000 los

In [6]:
# OR Gate prediction
print(or_obj.name, "\n")

test_data = np.array([ [0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 1] ])

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 1, 1, 1])

(true_list, false_list) = or_obj.accuracy1(test_xdata, test_tdata)
(true_list, false_list) = or_obj.accuracy2(test_data)

OR 

Accuracy =>  1.0
Accuracy =>  1.0


In [7]:
nand_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
nand_tdata = np.array([1, 1, 1, 0])

input_nodes = 2
hidden_1_nodes = 8
output_nodes = 1

lr = 1e-2  # learning rate
count = 10001  # iteration count

# OR Gate 객체 생성 및 training
nand_obj = LogicGate("NAND", nand_xdata, nand_tdata, input_nodes, hidden_1_nodes, output_nodes, lr, count)

nand_obj.train()

NAND object is created
Initial loss value =  3.8092367938398426
step =  0 loss value =  3.7611615664946485
step =  400 loss value =  2.2668957706886737
step =  800 loss value =  2.089890710695971
step =  1200 loss value =  1.7880612710983215
step =  1600 loss value =  1.3925638769484006
step =  2000 loss value =  1.0255873921426126
step =  2400 loss value =  0.7410437633126126
step =  2800 loss value =  0.5397546957720405
step =  3200 loss value =  0.40300196141554956
step =  3600 loss value =  0.31024859076152367
step =  4000 loss value =  0.24606733401518083
step =  4400 loss value =  0.20039000397591536
step =  4800 loss value =  0.16691905008882413
step =  5200 loss value =  0.14171337231273617
step =  5600 loss value =  0.1222611278981704
step =  6000 loss value =  0.10692137658147965
step =  6400 loss value =  0.0945937596815527
step =  6800 loss value =  0.08452147486164276
step =  7200 loss value =  0.07617153457764607
step =  7600 loss value =  0.06916032527899658
step =  8000

In [8]:
# NAND Gate prediction
print(nand_obj.name, "\n")

test_data = np.array([ [0, 0, 1], [0, 1, 1], [1, 0, 1], [1, 1, 0] ])

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 1, 1, 1, 0])

(true_list, false_list) = nand_obj.accuracy1(test_xdata, test_tdata)
(true_list, false_list) = nand_obj.accuracy2(test_data)

NAND 

Accuracy =>  1.0
Accuracy =>  1.0


In [9]:
xor_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
xor_tdata = np.array([0, 1, 1, 0])

input_nodes = 2
hidden_1_nodes = 8
output_nodes = 1

lr = 1e-2  # learning rate
count = 10001  # iteration count

# OR Gate 객체 생성 및 training
xor_obj = LogicGate("XOR", xor_xdata, xor_tdata, input_nodes, hidden_1_nodes, output_nodes, lr, count)

xor_obj.train()

XOR object is created
Initial loss value =  6.144681553337769
step =  0 loss value =  5.989518916382216
step =  400 loss value =  2.7668547390055798
step =  800 loss value =  2.763893265356324
step =  1200 loss value =  2.760222087083186
step =  1600 loss value =  2.75539810034975
step =  2000 loss value =  2.748792190188949
step =  2400 loss value =  2.739502456357439
step =  2800 loss value =  2.726256073436319
step =  3200 loss value =  2.7073064987622057
step =  3600 loss value =  2.680330498541676
step =  4000 loss value =  2.642369309976684
step =  4400 loss value =  2.5900485469799825
step =  4800 loss value =  2.5205782044554996
step =  5200 loss value =  2.4336331863424787
step =  5600 loss value =  2.3325750435867656
step =  6000 loss value =  2.223033652997834
step =  6400 loss value =  2.1098065145674822
step =  6800 loss value =  1.9948558443488542
step =  7200 loss value =  1.8773242287187566
step =  7600 loss value =  1.7547684744741425
step =  8000 loss value =  1.62490

In [10]:
# XOR Gate prediction
print(xor_obj.name, "\n")

test_data = np.array([ [0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 0] ])

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 1, 1, 0])

(true_list, false_list) = xor_obj.accuracy1(test_xdata, test_tdata)
(true_list, false_list) = xor_obj.accuracy2(test_data)

XOR 

Accuracy =>  1.0
Accuracy =>  1.0
